# Construyendo un sistema de análisis de sentimiento (III)

## Tarea final

## Enunciado

Usando un enfoque supervisado a partir del corpus hateval en castellano, enriquecido con recursos no supervisados, realice una herramienta que sea capaz de clasificar tweets en castellano.

En esta tarea se ha de entregar un fichero comprimido con todo el material utilizado en la práctica, así como una breve memoria.

## Implementación

### Paso 1: Descargando las librerías de nltk

Importamos la librería nltk. Si no la tenemos instalada la podemos instalar mediante pip

    pip install nltk
 
Una vez importada, descargamos los paquetes necesarios: en este caso, con los conjuntos "popular" y "spanish grammars" será suficiente. El "popular" descargará los paquetes mas populares, mientras que "spanish grammars" descargará los paquetes para realizar el procesamiento de texto en español

In [1]:
import nltk


nltk.download('popular')
nltk.download('spanish_grammars')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/almu/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk

True

### Paso 2: Cargando el corpus

Cargamos el corpus de entrenamiento desde el fichero "test_es.tsv", y el corpus de prueba desde el fichero "trial_es.tsv". Este fichero utiliza un formato csv separado por tabulaciones. Como columnas cargamos "text", correspondiente al texto del tuit; y "HS", correspondiente a las etiquetas binarias de odio/no_odio

Para ello utilizaremos la librería pandas, con el método `read_csv()`, indicando en sus parámetros el tipo de separador y el número de filas a leer. Esto nos cargará las 10 filas en un dataframe (al que llamaremos `corpus_df`) con las mismas columnas que en el fichero original, etiquetadas por sus respectivos nombres. 

Una vez cargado, mostraremos el dataframe para comprobar que se ha cargado correctamente

In [28]:
import pandas as pd

corpus_df_train = pd.read_csv("HateEval/train_es.tsv", sep="\t", usecols =["text", "HS"])
labels_train = pd.read_csv("HateEval/train_es.tsv", delimiter='\t', usecols=["HS"])


corpus_df_eval = pd.read_csv("HateEval/trial_es.tsv", sep="\t", usecols =["text", "HS"])
labels_eval = pd.read_csv("HateEval/trial_es.tsv", delimiter='\t', usecols=["HS"])

print(corpus_df_train)
print(corpus_df_eval)

                                                   text  HS
0     Easyjet quiere duplicar el número de mujeres p...   1
1     El gobierno debe crear un control estricto de ...   1
2     Yo veo a mujeres destruidas por acoso laboral ...   0
3     — Yo soy respetuoso con los demás, sólamente l...   0
4     Antonio Caballero y como ser de mal gusto e ig...   0
...                                                 ...  ..
4464  @miriaan_ac @Linaveso_2105 @HumildesSquad_ CÁL...   1
4465  @IvanDuque presidente en Cúcuta , tenemos prob...   1
4466              - Callaté Visto Que Te Dejo En Puta🎤🎶   0
4467  -¿porque los hombres se casan con las mujeres?...   1
4468  — No hay nada más lento que un caracol. — Cáll...   0

[4469 rows x 2 columns]
                                                 text  HS
0   @ian_delaCalva @IrantzuVarela @pikaramagazine ...   0
1   NINGUNA MUJER ES 'PUTA' ❗❗❗ https://t.co/cV0CQ...   0
2   Editar, además de complicado, es lo que hace d...   0
3   Bien joder una puta

### Paso 3: Separando el texto en tokens y clasificando

En este paso, realizamos la clasificación de las palabras en función de su categoría gramatical.

Antes de realizar la clasificación, separamos el texto de cada tuit en tokens. Para ello, utilizamos la clase `TweetTokenizer`, especializada en la extracción de tokens de tuits. Con esto, transformaremos en campo `'text'` de cada tuit en una lista de tokens, lo cual nos facilitará dicha clasificación

Tras esto, pasaremos cada token a minúsculas, eliminaremos stopwords y tokens no alfanuméricos (hashtag, citas, enlaces...) y los añadiremos a una lista de palabras.


Finalmente, aplicaremos el POS Tagger, que generará una columna añadiendo la categoría gramatical de cada palabra

In [29]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

stopwords.words('spanish')

cleantokens = []
tweet_tokenizer = TweetTokenizer()

for index, tweet in corpus_df_eval.iterrows():
    for word in tweet_tokenizer.tokenize(tweet['text']):
        word = word.lower()
        if word.isalnum() and word not in stopwords.words('spanish'):
            cleantokens.append(word)

## Ejercicio 2: Caracterización gramatical

### Paso 1: Obtener tokens del dataframe

De nuevo, tokenizamos el campo "text" del dataframe, transformando dicha columna en una lista de palabras de cada tuit. Repetimos el mismo proceso que en el ejercicio anterior, aplicando el `TweetTokenizer()`, pasando palabras a minúsculas, y eliminando stopwords y palabras con caracteres no alfanuméricos

Aplicamos el proceso con ambos conjuntos: de entrenamiento y de prueba

In [30]:
def token_split(wordlist: pd.DataFrame):
    token_list = tweet_tokenizer.tokenize(wordlist)
    
    words_no_stop = [word.lower() for word in token_list if word not in stopwords.words('spanish') and word.isalnum()]
    return words_no_stop

corpus_filtered_train = corpus_df_train.copy()
corpus_filtered_train['text'] = corpus_df_train['text'].apply(token_split)

corpus_filtered_eval = corpus_df_eval.copy()
corpus_filtered_eval['text'] = corpus_df_eval['text'].apply(token_split)

print(corpus_filtered_train)
print(corpus_filtered_eval)

                                                   text  HS
0     [easyjet, quiere, duplicar, número, mujeres, p...   1
1     [el, gobierno, debe, crear, control, estricto,...   1
2     [yo, veo, mujeres, destruidas, acoso, laboral,...   0
3     [yo, respetuoso, demás, sólamente, recuerdo, y...   0
4     [antonio, caballero, ser, mal, gusto, ignorant...   0
...                                                 ...  ..
4464                          [cállateeee, zorra, ahre]   1
4465  [presidente, cúcuta, problemas, venezolanos, d...   1
4466          [callaté, visto, que, te, dejo, en, puta]   0
4467  [hombres, casan, mujeres, cabras, saben, frega...   1
4468  [no, lento, caracol, cállate, hijo, puta, dice...   0

[4469 rows x 2 columns]
                                                 text  HS
0                        [oye, molestas, puta, madre]   0
1                          [ninguna, mujer, es, puta]   0
2   [editar, además, complicado, hace, merezca, pe...   0
3   [bien, joder, puta,

### Entrenamiento del POSTagger

Para ahorrar tiempo de cómputo, en lugar de utilizar el StanfordPOSTagger, utilizaremos el cess_esp.
Este provee de un corpus etiquetado, basado en artículos de periódicos. Utilizaremos este corpus para entrenar el UnigramTagger, el cual generará el POSTagger

Para el conjunto de etiquetas, se necesita el fichero `universal_tagset-ES.map`, provisto en el directorio.
Este fichero se ha de copiar en el directorio `~/nltk_data/taggers/universal_tagset/`

    cp universal_tagset-ES.map ~/nltk_data/taggers/universal_tagset/

In [31]:
from nltk.corpus import cess_esp
from nltk import UnigramTagger

cess_esp._tagset = 'universal_tagset-ES'
oraciones = cess_esp.tagged_sents(tagset='universal')
print(oraciones[0])

spanish_postagger = UnigramTagger(oraciones)

[('El', 'DET'), ('grupo', 'NOUN'), ('estatal', 'ADJ'), ('Electricité_de_France', 'NOUN'), ('-Fpa-', '.'), ('EDF', 'NOUN'), ('-Fpt-', '.'), ('anunció', 'VERB'), ('hoy', 'ADV'), (',', '.'), ('jueves', 'X'), (',', '.'), ('la', 'DET'), ('compra', 'NOUN'), ('del', 'ADP'), ('51_por_ciento', 'NUM'), ('de', 'ADP'), ('la', 'DET'), ('empresa', 'NOUN'), ('mexicana', 'ADJ'), ('Electricidad_Águila_de_Altamira', 'NOUN'), ('-Fpa-', '.'), ('EAA', 'NOUN'), ('-Fpt-', '.'), (',', '.'), ('creada', 'ADJ'), ('por', 'ADP'), ('el', 'DET'), ('japonés', 'ADJ'), ('Mitsubishi_Corporation', 'NOUN'), ('para', 'ADP'), ('poner_en_marcha', 'VERB'), ('una', 'DET'), ('central', 'NOUN'), ('de', 'ADP'), ('gas', 'NOUN'), ('de', 'ADP'), ('495', 'X'), ('megavatios', 'NOUN'), ('.', '.')]


### Aplicación del POSTagger

Se aprecia algo de menor precisión respecto al StanfordPOSTagger, pero sin suponer una reducción grave de precisión. A cambio, el tiempo de cómputo de reduce considerablemente, a pocos segundos.

Adaptamos la función utilizada en la tarea anterior al nuevo POSTagger

In [32]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
#from nltk.tag.stanford import StanfordPOSTagger

#spanish_postagger = StanfordPOSTagger('./StanfordTagger/spanish.tagger', './StanfordTagger/stanford-postagger.jar', encoding='utf8')

#penn_to_wn = {"a": wn.ADJ, "r": wn.ADV, "n": wn.NOUN, "v": wn.VERB}
penn_to_wn = {"ADJ": wn.ADJ, "ADV": wn.ADV, "NOUN": wn.NOUN, "VERB": wn.VERB, None: wn.NOUN}

def filter_categories(row: pd.DataFrame, category_list: list):
    tagged_words = spanish_postagger.tag(row['text'])
    
    tokens_filtered = []
    postags = []
    
    for elem in tagged_words:
        if elem[1] in category_list:
            tokens_filtered.append(elem[0])
            postags.append((elem[0], penn_to_wn[elem[1]]))
    
    row['text'] = tokens_filtered
    
    # Replace Penn tags with wn equivalents
    row['pos_tags'] = postags
        
    return row

corpus_gramfiltered_train = corpus_filtered_train.copy().apply(filter_categories, args=[penn_to_wn.keys()], axis=1)
corpus_gramfiltered_eval = corpus_filtered_eval.copy().apply(filter_categories, args=[penn_to_wn.keys()], axis=1)
print(corpus_gramfiltered_eval)
print(corpus_gramfiltered_train)

                                                 text  HS  \
0                        [oye, molestas, puta, madre]   0   
1                                   [mujer, es, puta]   0   
2   [editar, además, complicado, hace, merezca, pe...   0   
3             [bien, puta, alegría, mereces, pequeña]   0   
4   [política, levanta, sesión, hijos, puta, manda...   0   
..                                                ...  ..   
95  [imagen, pokemon, bailando, machupichu, hoy, c...   0   
96  [ciudadano, ruso, denis, yakovlev, facilitó, f...   0   
97  [toy, feliz, voy, comer, empanadas, árabes, vi...   0   
98  [barrios, pueblo, solidario, caos, acogida, in...   0   
99  [noche, moritos, reggaeton, arabe, dejan, desc...   0   

                                             pos_tags  
0    [(oye, v), (molestas, n), (puta, n), (madre, n)]  
1                    [(mujer, n), (es, v), (puta, n)]  
2   [(editar, n), (además, r), (complicado, a), (h...  
3   [(bien, r), (puta, n), (alegría, n), (m

## Clasificación de sentimientos

### Obtención de sentimientos

Creamos la función `get_sentiment()` que permite obtener los sentimientos asociados a cada palabra.
Para ello, obtenemos su lema, su synset y su senti_synset, y generamos una tupla con el nombre del synset, la puntuación del sentisynset positivo y negativo, y la puntuación global.

Para obtener el synset, puesto que WordNet per se no soporta el idioma español, añadimos la extensión OpenMultiWordnet (omw)

Opcionalmente, en caso de no encontrar el synset, se puede traducir el término al inglés y buscarlo de nuevo en WordNet. Aunque actualmente dicha opción está desactivada por motivos de coste temporal

In [33]:
from deep_translator import PonsTranslator # pip install deep_translator
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

nltk.download('omw')

translator = PonsTranslator(source='spanish', target='english')
def get_sentiment(word,tag):             
    #Synset es un tipo especial de interfaz simple que está presente en NLTK para buscar palabras en WordNet.
    #Las instancias #Synset son agrupaciones de palabras sinónimos que expresan el mismo concepto.
    #Algunas de las palabras tienen solo un Synset y otras tienen varios.
    synsets = wn.synsets(word, pos=tag, lang="spa")
    
    #if not synsets:
    #    try:
    #        translation = translator.translate(word, return_all=False)
    #        synsets = wn.synsets(translation, pos=tag, lang="eng")
    #    except:
    #        None

    if not synsets:
        return []
    
    # Tomamos la primera acepción, la más común
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name()) #y la buscamos en sentiwordnet

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

[nltk_data] Downloading package omw to /home/almu/nltk_data...
[nltk_data]   Package omw is already up-to-date!


### Ponderación de sentimientos

Utilizando la función anterior, realizamos la ponderación del sentimiento global de cada frase.
Obtenemos la ponderación positiva y negativa sumando las de cada palabra de la frase, y calculamos el valor final restando la puntuación negativa a la positiva.

Los resultados los almacenamos en una nueva columna llamada `senti_score`

In [34]:
senti_score = []

def calculate_sentiment(row: pd.DataFrame):
    pos=neg=0
    pos_val = row['pos_tags']
    
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    
    for score in senti_val:
        try:
            pos = pos + score[1]  #la puntuación positiva se almacena en la segunda posición
            neg = neg + score[2]  #la puntuación negativa se almacena en la tercera posición
        except:
            continue
    
    row['senti_score'] = pos - neg
                
    return row
    
corpus_sentiscore_train = corpus_gramfiltered_train.copy().apply(calculate_sentiment, axis=1)
print(corpus_sentiscore_train['senti_score'])

print(corpus_sentiscore_train.head)

corpus_sentiscore_eval = corpus_gramfiltered_eval.copy().apply(calculate_sentiment, axis=1)
print(corpus_sentiscore_eval['senti_score'])

print(corpus_sentiscore_eval)


0      -0.375
1      -0.250
2      -1.875
3      -0.625
4       0.250
        ...  
4464    0.000
4465    0.000
4466    0.000
4467    0.250
4468    0.125
Name: senti_score, Length: 4469, dtype: float64
<bound method NDFrame.head of                                                    text  HS  \
0     [easyjet, quiere, duplicar, número, mujeres, p...   1   
1     [gobierno, debe, crear, control, estricto, inm...   1   
2     [veo, mujeres, destruidas, acoso, laboral, cal...   0   
3     [respetuoso, sólamente, recuerdo, escoria, cul...   0   
4     [antonio, caballero, ser, mal, gusto, ignorant...   0   
...                                                 ...  ..   
4464                          [cállateeee, zorra, ahre]   1   
4465  [presidente, cúcuta, problemas, venezolanos, d...   1   
4466                       [callaté, visto, dejo, puta]   0   
4467  [hombres, casan, mujeres, cabras, saben, frega...   1   
4468  [no, lento, caracol, cállate, hijo, puta, dice...   0   

           

In [35]:
overall=[]

def tag_sentiments(data: pd.DataFrame):
    if data['senti_score']>= 0.05:
        overall = 'Positive'
    elif data['senti_score']<= -0.05:
        overall = 'Negative'
    else:
        overall= 'Neutral'
        
    data['Overall_Sentiment'] = overall

    return data
    
corpus_tagsents_train = corpus_sentiscore_train.copy().apply(tag_sentiments, axis=1)
print(corpus_tagsents_train.head)

corpus_tagsents_eval = corpus_sentiscore_eval.copy().apply(tag_sentiments, axis=1)
print(corpus_tagsents_eval.head)

<bound method NDFrame.head of                                                    text  HS  \
0     [easyjet, quiere, duplicar, número, mujeres, p...   1   
1     [gobierno, debe, crear, control, estricto, inm...   1   
2     [veo, mujeres, destruidas, acoso, laboral, cal...   0   
3     [respetuoso, sólamente, recuerdo, escoria, cul...   0   
4     [antonio, caballero, ser, mal, gusto, ignorant...   0   
...                                                 ...  ..   
4464                          [cállateeee, zorra, ahre]   1   
4465  [presidente, cúcuta, problemas, venezolanos, d...   1   
4466                       [callaté, visto, dejo, puta]   0   
4467  [hombres, casan, mujeres, cabras, saben, frega...   1   
4468  [no, lento, caracol, cállate, hijo, puta, dice...   0   

                                               pos_tags  senti_score  \
0     [(easyjet, n), (quiere, v), (duplicar, v), (nú...       -0.375   
1     [(gobierno, n), (debe, v), (crear, v), (contro...       -0.250 

## Preparando la bolsa de palabras
### Generando bolsa de nombres, adjetivos y adverbios

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus_sentiscore_str_train = corpus_sentiscore_train.copy()
corpus_sentiscore_str_eval = corpus_sentiscore_eval.copy()

# Transformamos la lista de palabras en frases 
corpus_sentiscore_str_train["text"] = corpus_sentiscore_str_train["text"].apply(lambda wordlist: ' '.join(wordlist))
corpus_sentiscore_str_eval["text"] = corpus_sentiscore_str_eval["text"].apply(lambda wordlist: ' '.join(wordlist))

# creamos la matriz
taghate_vectorizer_train = TfidfVectorizer()
taghate_vectorizer_eval = TfidfVectorizer()

# construimos vocabulario
vector_taghate_train = taghate_vectorizer_train.fit_transform(corpus_sentiscore_str_train["text"])
vector_taghate_eval = taghate_vectorizer_eval.fit_transform(corpus_sentiscore_str_eval["text"])

print(taghate_vectorizer_train.vocabulary_)
print(taghate_vectorizer_eval.vocabulary_)
print(vector_taghate_train)

{'easyjet': 4240, 'quiere': 10501, 'duplicar': 4202, 'número': 8766, 'mujeres': 8439, 'piloto': 9706, 'verás': 12894, 'aparcar': 866, 'avión': 1294, 'gobierno': 5665, 'debe': 3439, 'crear': 3129, 'control': 2984, 'estricto': 4864, 'inmigración': 6569, 'zonas': 13319, 'fronterizas': 5427, 'colombia': 2644, 'después': 3831, 'querrán': 10486, 'venir': 12842, 'masa': 7854, 'veo': 12853, 'destruidas': 3848, 'acoso': 280, 'laboral': 7099, 'callejero': 1936, 'depresión': 3675, 'debido': 3454, 'violación': 12986, 'sexual': 11532, 'maltrato': 7682, 'físico': 5499, 'conocí': 2885, 'suicidaron': 11909, 'tipo': 12239, 'comportamientos': 2767, 'machistas': 7576, 'vas': 12762, 'seguir': 11423, 'show': 11561, 'pobre': 9827, 'respetuoso': 10929, 'sólamente': 11972, 'recuerdo': 10709, 'escoria': 4689, 'culpa': 3292, 'claro': 2528, 'sé': 11970, 'tomas': 12309, 'antonio': 844, 'caballero': 1800, 'ser': 11494, 'mal': 7640, 'gusto': 5852, 'ignorante': 6310, 'vez': 12910, 'conductas': 2838, 'componen': 2764

## Entrenando los modelos

### Paso 1: Dividiendo las colecciones en subconjuntos

#### Generando subconjuntos para bolsa de nombres, adjetivos y adverbios

In [37]:
from sklearn.model_selection import train_test_split

# división del conjunto en entrenamiento y test

X_taghate_train, X_taghate_test, y_taghate_train, y_taghate_test = train_test_split(vector_taghate_train, labels_train,

                                                    stratify=labels_train,

                                                    test_size=0.2,

                                                    random_state=1234)

print(X_taghate_train)
print(y_taghate_train)
print(labels_train)

  (0, 11391)	0.24301419538212476
  (0, 591)	0.24301419538212476
  (0, 1181)	0.23170406952030634
  (0, 2507)	0.24301419538212476
  (0, 12684)	0.24301419538212476
  (0, 5477)	0.24301419538212476
  (0, 13382)	0.24301419538212476
  (0, 9927)	0.23170406952030634
  (0, 3611)	0.24301419538212476
  (0, 1226)	0.24301419538212476
  (0, 12925)	0.24301419538212476
  (0, 115)	0.22367940751474005
  (0, 182)	0.24301419538212476
  (0, 463)	0.24301419538212476
  (0, 12340)	0.24301419538212476
  (0, 8878)	0.1817243679237184
  (0, 4816)	0.1887345834956826
  (0, 13160)	0.1868100823623384
  (0, 6571)	0.11584100427923257
  (0, 8439)	0.13023650430446276
  (1, 987)	0.47086480771487943
  (1, 1533)	0.47086480771487943
  (1, 9819)	0.448950284446878
  (1, 13131)	0.4334016827436468
  (1, 4540)	0.2562790079002984
  :	:
  (3571, 9495)	0.3959235846751947
  (3571, 3918)	0.41524972591910725
  (3571, 9878)	0.20293714149354228
  (3571, 6825)	0.2637705579763081
  (3571, 3349)	0.16440818153004785
  (3572, 10041)	0.52260697

### Paso 2: Ajustando los modelos del clasificador

Utilizaremos el clasificador SVC, debido a que el MLP es incapaz de converger

#### Ajustando el modelo para la bolsa de nombres, adjetivos y adverbios

In [38]:
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report

clasificador_taghate_svc = svm.SVC(kernel='rbf', gamma='auto', C=300)
clasificador_taghate_svc.fit(X_taghate_train, np.ravel(y_taghate_train))
pred_y_taghate_svc = clasificador_taghate_svc.predict(X_taghate_test)

print("CCR: %f"%(clasificador_taghate_svc.score(X_taghate_test, y_taghate_test)))
print(classification_report(y_taghate_test, pred_y_taghate_svc))

CCR: 0.626398
              precision    recall  f1-score   support

           0       0.61      1.00      0.76       526
           1       0.97      0.10      0.17       368

    accuracy                           0.63       894
   macro avg       0.79      0.55      0.47       894
weighted avg       0.76      0.63      0.52       894



## Probando los modelos en datos reales

Una vez convergidos los modelos, los aplicamos sobre datos reales, utilizando para ello el conjunto de evaluación (eval). Observamos una precisión total en la clase de odio, y una precisión del 50% en la clase de no-odio.

In [40]:
eval_taghate = taghate_vectorizer_train.transform(corpus_sentiscore_str_eval['text'])
evalPredict_taghate_svc = clasificador_taghate_svc.predict(eval_taghate)

print("CCR: %f"%(clasificador_taghate_svc.score(eval_taghate, labels_eval)))
print(classification_report(labels_eval, evalPredict_taghate_svc))
print(evalPredict_taghate_svc)

CCR: 0.530000
              precision    recall  f1-score   support

           0       0.52      1.00      0.68        50
           1       1.00      0.06      0.11        50

    accuracy                           0.53       100
   macro avg       0.76      0.53      0.40       100
weighted avg       0.76      0.53      0.40       100

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Conclusiones

El TweetTokenizer es una gran herramienta para extraer tokens de mensajes provenientes de Twitter. Mejora la identificación de tokens con características especiales, como hashtags, menciones, URLs... Esto permite realizar un filtrado posterior de forma mas precisa, sin necesidad de recurrir a expresiones regulares.

Respecto al POS Tagging, hemos descubierto que es una herramienta bastante poderosa para realizar un análisis gramatical. Por contra, la clasificación gramatical no es perfecta, y el tiempo de cómputo es bastante alto.

Tras realizar el filtrado de nombres, adjetivos, verbos y adverbios, hemos aplicado un análisis de sentimientos no supervisado, generando una ponderación según el sentimiento sea positivo o negativo. Esto lo hemos utilizado para regenerar la columna "HS", correspondiente al discurso de odio, etiquetando como odio aquellos tuits que tengan una ponderación inferior a -0.125.

En esta ocasión, se aprecia un gran aumento de precisión respecto a la tarea 7b, y un soporte mucho mas equilibrado entre ambas clases.